# Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor,BaggingRegressor
from sklearn.linear_model import LinearRegression,Ridge,ElasticNet,SGDRegressor
from xgboost import XGBRegressor
from sklearn import metrics

# Load Data

In [2]:
car = pd.read_csv('CarPrice_Assignment.csv')
car.reset_index(drop=True,inplace=True)
pd.set_option('display.max_columns', None)

In [3]:
car.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
car.drop(columns='car_ID',inplace=True)

In [5]:
def displaying_unique_values(type):
    for col in car.select_dtypes(include=type):
        print("*" * 75,col,"*"*75)
        print(list(set(car[col])))
        print()

In [6]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   symboling         205 non-null    int64  
 1   CarName           205 non-null    object 
 2   fueltype          205 non-null    object 
 3   aspiration        205 non-null    object 
 4   doornumber        205 non-null    object 
 5   carbody           205 non-null    object 
 6   drivewheel        205 non-null    object 
 7   enginelocation    205 non-null    object 
 8   wheelbase         205 non-null    float64
 9   carlength         205 non-null    float64
 10  carwidth          205 non-null    float64
 11  carheight         205 non-null    float64
 12  curbweight        205 non-null    int64  
 13  enginetype        205 non-null    object 
 14  cylindernumber    205 non-null    object 
 15  enginesize        205 non-null    int64  
 16  fuelsystem        205 non-null    object 
 1

In [7]:
#displaying unique values of category values
displaying_unique_values('object')


*************************************************************************** CarName ***************************************************************************
['alfa-romero stelvio', 'volvo 264gl', 'toyota starlet', 'porsche boxter', 'dodge d200', 'alfa-romero Quadrifoglio', 'bmw 320i', 'vw rabbit', 'peugeot 304', 'audi 100 ls', 'toyota celica gt liftback', 'maxda rx3', 'toyota corona', 'nissan kicks', 'dodge colt hardtop', 'toyota mark ii', 'toyota cressida', 'toyota corolla 1200', 'volvo 144ea', 'honda civic 1300', 'vw dasher', 'toyota corolla tercel', 'volvo 245', 'buick electra 225 custom', 'dodge colt (sw)', 'mitsubishi pajero', 'nissan juke', 'isuzu MU-X', 'volkswagen rabbit custom', 'chevrolet vega 2300', 'saab 99e', 'honda civic 1500 gl', 'subaru', 'peugeot 505s turbo diesel', 'alfa-romero giulia', 'volvo 244dl', 'volvo 246', 'nissan nv200', 'porsche macan', 'audi fox', 'honda accord lx', 'toyota tercel', 'audi 100ls', 'nissan leaf', 'honda accord cvcc', 'nissan latio', 'bmw x

In [8]:
displaying_unique_values('number')

*************************************************************************** symboling ***************************************************************************
[0, 1, 2, 3, -2, -1]

*************************************************************************** wheelbase ***************************************************************************
[102.4, 102.9, 104.5, 104.3, 97.0, 95.1, 100.4, 97.3, 107.9, 108.0, 95.7, 109.1, 96.6, 96.1, 95.3, 96.9, 93.1, 93.0, 86.6, 96.3, 88.6, 88.4, 89.5, 91.3, 93.3, 93.7, 94.3, 94.5, 96.5, 96.0, 95.9, 99.8, 99.4, 99.5, 101.2, 103.5, 103.3, 105.8, 102.0, 98.8, 104.9, 106.7, 110.0, 102.7, 112.0, 113.0, 97.2, 115.6, 99.2, 114.2, 98.4, 99.1, 120.9]

*************************************************************************** carlength ***************************************************************************
[180.2, 141.1, 181.5, 144.6, 150.0, 183.5, 183.1, 155.9, 156.9, 157.3, 158.8, 157.1, 159.1, 157.9, 162.4, 163.4, 158.7, 165.3, 166.8, 167.5, 168.8, 16

In [9]:
# separating true categorical values  and true numeric varaibales
true_categorical = car.select_dtypes(include = 'object').columns
true_numeric= car.select_dtypes(include = 'number').columns

In [10]:
true_categorical

Index(['CarName', 'fueltype', 'aspiration', 'doornumber', 'carbody',
       'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber',
       'fuelsystem'],
      dtype='object')

In [11]:
true_numeric

Index(['symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight',
       'curbweight', 'enginesize', 'boreratio', 'stroke', 'compressionratio',
       'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price'],
      dtype='object')

# checking null values

In [12]:
car.isnull().sum()

symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

# Encoding

In [13]:
for col in true_categorical:
    car[col] = car[col].astype('category').cat.codes

# Independent and dependent variable

In [14]:
X= car.drop(columns='price')
y = car.price

# Feature selection

In [15]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
def select_categorical_features(true_categorical):
    significant_feature = []
    for col in true_categorical:
        formula = f"{'price'}  ~ C({col})"
        model = ols(formula,data = car).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        p_value = anova_table["PR(>F)"].iloc[0]
        if p_value <0.05:
            significant_feature.append(col)
        else:
            print(col)
    return significant_feature
            

In [16]:
significant_feature = select_categorical_features(true_categorical)

fueltype
doornumber


In [17]:
significant_feature

['CarName',
 'aspiration',
 'carbody',
 'drivewheel',
 'enginelocation',
 'enginetype',
 'cylindernumber',
 'fuelsystem']

In [18]:
car.head()

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,2,1,0,1,0,2,0,88.6,168.8,64.1,48.8,2548,0,2,130,5,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,3,1,0,1,0,2,0,88.6,168.8,64.1,48.8,2548,0,2,130,5,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,1,1,0,1,2,2,0,94.5,171.2,65.5,52.4,2823,5,3,152,5,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,4,1,0,0,3,1,0,99.8,176.6,66.2,54.3,2337,3,2,109,5,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,5,1,0,0,3,0,0,99.4,176.6,66.4,54.3,2824,3,1,136,5,3.19,3.40,8.0,115,5500,18,22,17450.0


In [19]:
final_cols = significant_feature+list(true_numeric[:-1])

# Train_test_split

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X.loc[:,final_cols],y,test_size=0.2,random_state=42)

In [21]:
X_train.shape

(164, 22)

In [22]:
#this function returns the score all 8 algo and choosing the best among them

def scores(X,y):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=101)
    
    models = dict(zip('RandomForest   GradientBoosting Xgboost lr ridge'.split(),
                      
                      [RandomForestRegressor(),
                      GradientBoostingRegressor(),
                       XGBRegressor(),LinearRegression(),Ridge()
                      ])
                 )
    algo_scores = pd.DataFrame()
    algo_scores['model_name']=list(models.keys())
    train_score = []
    test_score = []
    for _,estimator in models.items():
        estimator.fit(X_train,y_train)
        
        y_train_preds=estimator.predict(X_train)
        
        y_test_preds=estimator.predict(X_test)
        train_r2 = metrics.r2_score(y_train,y_train_preds)
        test_r2 = metrics.r2_score(y_test,y_test_preds)
        train_score.append(float(f'{train_r2:.2f}'))
        test_score.append(float(f'{test_r2:.2f}'))
    algo_scores["Train_scores"]=train_score
    algo_scores["Test_scores"]=test_score
    algo_scores.sort_values(by='Test_scores',ascending=False,inplace=True)
     
    return algo_scores
        
    
    
performances= scores(X.loc[:,final_cols],y)





In [23]:
performances

,model_name,Train_scores,Test_scores
0,RandomForest,0.99,0.91
1,GradientBoosting,1.00,0.90
2,Xgboost,1.00,0.90
3,lr,0.90,0.81
4,ridge,0.90,0.81


**I am going to choose Linear Regression Model**